In [1]:
import requests
import numpy as np
import random
import math

In [2]:
url = 'https://www.csie.ntu.edu.tw/~htlin/course/ml21fall/hw6/hw6_train.dat'
content = requests.get(url).content
content = content.decode('utf-8')

with open('./hw6_train.txt', mode='w') as f:
  for line in content:
    f.write(line)


In [3]:
url = 'https://www.csie.ntu.edu.tw/~htlin/course/ml21fall/hw6/hw6_test.dat'
content = requests.get(url).content
content = content.decode('utf-8')

with open('./hw6_test.txt', mode='w') as f:
  for line in content:
    f.write(line)

# Load Data

In [4]:
data = np.loadtxt('hw6_train.txt')
X = data[:,:-1]
y = data[:,-1]

In [5]:
data2 = np.loadtxt('hw6_test.txt')
X2 = data2[:,:-1]
y2 = data2[:,-1]

In [7]:
ITER = 20

In [8]:
def stump(x, y, u):
    '''x: one dimension feature'''
    N, dim = X.shape
    feature_x_sorted = np.sort(x)
    thetas = [ feature_x_sorted[0]-1 ] # left-most
    for n in range(N-2):
        thetas.append( (feature_x_sorted[n]+feature_x_sorted[n+1])/2 )
    thetas.append( feature_x_sorted[-1]+1 ) # right-most

    best_err = 1.01 # initialize with "101% err"
    for theta in thetas:
        y_pred = np.sign(x-theta) # Nx1 array with {-1,1}
        err_pos = np.sum( (y != 1*y_pred)*u ) / N
        err_neg = np.sum( (y != -1*y_pred)*u ) / N
        err = err_neg if err_neg < err_pos else err_pos
        s = -1 if err_neg < err_pos else 1
        
        if err < best_err or (err==best_err and random.random()>0.5):
            best_s = s
            best_theta = theta
            best_err = err
    
    return best_s, best_theta, best_err

In [9]:
def stump_mult(X, y, u):
    N, dim = X.shape
    best_err = 1.01 # initialize as "101% wrong"
    for d in range(dim):
        s_d, theta_d, err_d = stump(X[:,d], y, u)
        if err_d < best_err or (err_d==best_err and random.random()>0.5):
            best_d = d
            best_s = s_d
            best_theta = theta_d
            best_err = err_d

    result = {'d': best_d, 's': best_s, 'theta': best_theta, 'err': best_err}
    return result

In [10]:
def calc_E_Gt(X, y, params, alphas, uniform=False):
    N,dim = X.shape
    T = len(alphas)
    G = np.zeros(N)
    for t in range(T):
        d, s, theta = params[t]['d'], params[t]['s'], params[t]['theta']
        x_d = X[:,d]
        gt = s*np.sign(x_d-theta)
        # G = G + alphas[t]*gt
        if uniform:
            G = G + gt
        elif not uniform:
            G = G + (alphas[t]*gt)
            # print(G)
    return np.sum(np.sign(G)!=y) / len(y)

In [ ]:
# ADA Boost
N,dim = X.shape
us = [np.ones(N)/N]  # u_0 [1/N, 1/N, ... 1/N]
alphas = []
params = []

e_Gs = []
e_gs = []

for t in range(ITER):
# for t in range(ITER):
    print(t)
    param = stump_mult(X, y, us[t])
    params.append( param )

    epsilon_t = param['err']*N / np.sum(us[t])
    diamond_t = math.sqrt( (1-epsilon_t)/epsilon_t )

    s, theta, x_d = param['s'], param['theta'], X[:,param['d']]
    y_pred = s * np.sign( x_d-theta )

    # if correct, u <-- u/diamond_t. Else, u <-- u*diamond_t
    us.append( np.where( y==y_pred, us[-1]/diamond_t, us[-1]*diamond_t ) )
    alphas.append( math.log(diamond_t) )

    # calculate E_in_G
    e_Gs.append( calc_E_Gt(X, y, params, alphas, uniform=False) )
    e_gs.append( np.sum(y!=y_pred)/N )

In [14]:
for p in params:
    print(p)

{'d': 9, 's': -1, 'theta': 0.44824087255, 'err': 0.000374}
{'d': 0, 's': 1, 'theta': 2.6792002063, 'err': 0.00038290128340678914}
{'d': 0, 's': -1, 'theta': -1.51380683245, 'err': 0.00037746709273849075}
{'d': 5, 's': 1, 'theta': -1.24708306405, 'err': 0.00033573752370713}
{'d': 0, 's': 1, 'theta': 3.8257959154, 'err': 0.0003675637114235907}
{'d': 8, 's': 1, 'theta': -0.5865877720499999, 'err': 0.00035731593521318026}
{'d': 0, 's': -1, 'theta': -2.25328903845, 'err': 0.000354639661311784}
{'d': 7, 's': 1, 'theta': -0.67601466975, 'err': 0.00033990543712253044}
{'d': 5, 's': 1, 'theta': 2.94708224915, 'err': 0.000361863456536178}
{'d': 8, 's': -1, 'theta': 0.5180065872999999, 'err': 0.0003387973450474271}
{'d': 0, 's': 1, 'theta': 1.4465262918, 'err': 0.00035432854349213845}
{'d': 9, 's': -1, 'theta': 0.79359644275, 'err': 0.0003492977711519339}
{'d': 2, 's': 1, 'theta': 3.8516220379, 'err': 0.0003342391343917704}
{'d': 9, 's': 1, 'theta': -0.647044464, 'err': 0.00031833565749417477}
{'

# Solutions

In [ ]:
print(f'[Q11] Ein_g1={e_gs[0]}')

[Q11] Ein_g1=0.374


In [ ]:
print(f'[Q12] max Ein={max(e_gs)}')

[Q12] max Ein=0.563


In [ ]:
for i, e_in in enumerate(e_Gs):
    if e_in <= 0.05: break
print(f'[Q13] {i}')

[Q13] 364


In [ ]:
e_out_gs = []
e_out_Gs = []
for t in range(len(alphas)):
    d, s, theta = params[t]['d'], params[t]['s'], params[t]['theta']
    x_d = X2[:,d]
    param = params[t]
    y2_pred = s*np.sign(x_d-theta)
    e_out_gs.append( np.sum(y2!=y2_pred)/len(y2) )
    e_out_Gs.append( calc_E_Gt(X2, y2, params[:t], alphas[:t]) )

In [ ]:
print(f'[Q14] Eout_g1={e_out_gs[0]}')

[Q14] Eout_g1=0.455


In [ ]:
print(f'[Q16] Eout_G500={e_out_Gs[-1]}')

[Q16] Eout_G500=0.181


In [ ]:
e_out_gs = []
e_out_Gs = []
for t in range(len(alphas)):
    d, s, theta = params[t]['d'], params[t]['s'], params[t]['theta']
    x_d = X2[:,d]
    param = params[t]
    y2_pred = s*np.sign(x_d-theta)
    e_out_gs.append( np.sum(y2!=y2_pred)/len(y2) )
    e_out_Gs.append( calc_E_Gt(X2, y2, params[:t], alphas[:t], uniform=True) )

In [ ]:
print(f'[Q15] Eout_Gunif={e_out_Gs[-1]}')

[Q15] Eout_Gunif=0.235


In [ ]:
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!pip install pypandoc

In [ ]:
!jupyter nbconvert --to PDF "HTML_HW6.ipynb"

[NbConvertApp] Converting notebook HTML_HW6.ipynb to PDF
[NbConvertApp] Writing 40479 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 31977 bytes to HTML_HW6.pdf
